In [1]:
import pandas as pd
from selenium import webdriver
import time as t
from bs4 import BeautifulSoup
import numpy as np
import requests

In [2]:
url = 'http://br.advfn.com/bolsa-de-valores/bovespa/vale-on-VALE3/historico/mais-dados-histprocos'
url2 = 'http://br.advfn.com/bolsa-de-valores/bovespa/vale-on-VALE3/historico/mais-dados-historicos?current=0&Date1=28/01/19&Date2=28/04/22' # inicio
url3 = 'https://br.advfn.com/bolsa-de-valores/bovespa/vale-on-VALE3/historico/mais-dados-historicos?current=21&Date1=28/01/17&Date2=28/04/22' # fim
url4 = 'https://br.advfn.com/bolsa-de-valores/bovespa/petrobras-pn-PETR4/historico/mais-dados-historicos?current=0&Date1=28/01/19&Date2=28/04/22' # inicio
url5 = 'https://br.advfn.com/bolsa-de-valores/bovespa/petrobras-pn-PETR4/historico/mais-dados-historicos?current=21&Date1=28/01/19&Date2=28/04/22' # fim
url6 = 'https://br.advfn.com/bolsa-de-valores/bovespa/banco-do-brasil-on-BBAS3/historico/mais-dados-historicos?current=0&Date1=01/02/17&Date2=02/05/22' # inicio
url6 = 'https://br.advfn.com/bolsa-de-valores/bovespa/banco-do-brasil-on-BBAS3/historico/mais-dados-historicos?current=21&Date1=01/02/17&Date2=02/05/22' # fim

In [3]:
browser = webdriver.Chrome()
browser.get(url2)
t.sleep(3)
html = browser.page_source
browser.close()
bs = BeautifulSoup(html, 'html.parser')
linhas = bs.find_all('tr', attrs={'class': 'result'})
print(linhas)

[<tr class="result">
<td>28 Abr 2022</td>
<td class="Numeric">84,14</td>
<td class="Numeric PriceTextUp">1,86</td>
<td class="Numeric PriceTextUp">2,26%</td>
<td class="Numeric">85,02</td>
<td class="Numeric">85,55</td>
<td class="Numeric">80,83</td>
<td class="Numeric">56.032.800</td>
</tr>, <tr class="result">
<td>27 Abr 2022</td>
<td class="Numeric">82,28</td>
<td class="Numeric PriceTextUp">4,63</td>
<td class="Numeric PriceTextUp">5,96%</td>
<td class="Numeric">80,46</td>
<td class="Numeric">82,70</td>
<td class="Numeric">80,15</td>
<td class="Numeric">49.408.800</td>
</tr>, <tr class="result">
<td>26 Abr 2022</td>
<td class="Numeric">77,65</td>
<td class="Numeric PriceTextDown">-1,63</td>
<td class="Numeric PriceTextDown">-2,06%</td>
<td class="Numeric">79,08</td>
<td class="Numeric">79,32</td>
<td class="Numeric">77,65</td>
<td class="Numeric">37.069.400</td>
</tr>, <tr class="result">
<td>25 Abr 2022</td>
<td class="Numeric">79,28</td>
<td class="Numeric PriceTextDown">-1,04</t

In [4]:
i = 0
data = []
while i < len(linhas):
    data.append(linhas[i].text)
    i +=1

In [5]:
j=0
datanew = []
while j<len(data):
    datanew.append(data[j].split('\n'))
    j +=1

In [6]:
i = 1
for k in range(len(datanew) -1):
    for i in range(len(datanew[k])-1):
        print(datanew[k][i])


28 Abr 2022
84,14
1,86
2,26%
85,02
85,55
80,83
56.032.800

27 Abr 2022
82,28
4,63
5,96%
80,46
82,70
80,15
49.408.800

26 Abr 2022
77,65
-1,63
-2,06%
79,08
79,32
77,65
37.069.400

25 Abr 2022
79,28
-1,04
-1,29%
77,94
79,61
77,54
38.943.700

22 Abr 2022
80,32
-4,98
-5,84%
83,25
83,40
80,00
39.058.200

21 Abr 2022
85,30
0,00
+0,00%
86,61
86,67
84,40
0

20 Abr 2022
85,30
-2,30
-2,63%
86,61
86,67
84,40
29.635.700

19 Abr 2022
87,60
-2,61
-2,89%
89,69
89,69
86,84
34.550.900

18 Abr 2022
90,21
-1,88
-2,04%
91,61
92,00
90,01
14.812.600

15 Abr 2022
92,09
0,00
+0,00%
93,31
94,00
91,36
0

14 Abr 2022
92,09
-1,29
-1,38%
93,31
94,00
91,36
24.666.100

13 Abr 2022
93,38
0,00
0,0%
93,49
94,38
92,81
23.674.700

12 Abr 2022
93,38
-0,62
-0,66%
95,10
95,61
93,24
16.822.600

11 Abr 2022
94,00
-0,95
-1,0%
93,81
94,71
92,92
16.515.800

08 Abr 2022
94,95
-2,20
-2,26%
96,64
97,40
94,62
26.887.400

07 Abr 2022
97,15
0,65
0,67%
95,50
97,69
95,14
19.946.600

06 Abr 2022
96,50
1,37
1,44%
95,33
96,72
95,33
22.615

In [7]:
len(datanew[0])

10

In [57]:
print(data[1].split('\n'))

['', '06 Mar 2019', '48,05', '1,31', '2,8%', '47,43', '48,05', '47,19', '15.244.000', '']


In [105]:
datanew[3][1]

'04 Mar 2019'

In [8]:
l = 1 
dias = []
fechamento = []
variacao = []
variacao_p = []
abertura = []
maxima = []
minima = []
volume = []


In [9]:
while l < len(datanew):
    dias.append(datanew[l][1])
    fechamento.append(datanew[l][2])
    variacao.append(datanew[l][3])
    variacao_p.append(datanew[l][4])
    abertura.append(datanew[l][5])
    maxima.append(datanew[l][6])
    minima.append(datanew[l][7])
    volume.append(datanew[l][8])
    l += 1

In [127]:
volume

['15.244.000',
 '0',
 '0',
 '29.872.100',
 '17.877.800',
 '15.004.800',
 '10.362.900',
 '16.801.500',
 '17.580.700',
 '21.401.300',
 '25.493.000',
 '20.523.500',
 '18.777.400',
 '0',
 '27.877.700',
 '24.055.400',
 '38.153.200',
 '36.398.200',
 '30.697.300',
 '48.169.600',
 '31.403.200',
 '29.257.400',
 '21.995.000',
 '35.869.500',
 '21.286.700',
 '39.814.600',
 '67.891.800',
 '68.335.900',
 '183.534.500']

In [23]:
df = pd.DataFrame({
        'Data': dias,
        'Fechamento': fechamento,
        'Variacao': variacao,
        'Variacao_%': variacao_p,
        'Abertura': abertura,
        'Maxima': maxima,
        'Minima': minima,
    })

In [24]:
df

,Data,Fechamento,Variacao,Variacao_%,Abertura,Maxima,Minima
0,27 Abr 2022,"82,28","4,63","5,96%","80,46","82,70","80,15"
1,26 Abr 2022,"77,65","-1,63","-2,06%","79,08","79,32","77,65"
2,25 Abr 2022,"79,28","-1,04","-1,29%","77,94","79,61","77,54"
3,22 Abr 2022,"80,32","-4,98","-5,84%","83,25","83,40","80,00"
4,21 Abr 2022,"85,30","0,00","+0,00%","86,61","86,67","84,40"
...,...,...,...,...,...,...,...
59,03 Fev 2022,"85,75","-0,04","-0,05%","86,30","86,78","85,32"
60,02 Fev 2022,"85,79","0,48","0,56%","85,50","86,29","84,72"
61,01 Fev 2022,"85,31","4,44","5,49%","81,50","85,33","81,37"
62,31 Jan 2022,"80,87","-2,79","-3,33%","82,94","82,94","80,87"


In [25]:
df['Fechamento'] = df['Fechamento'].map(lambda t: t.replace(',', '.'))
df['Fechamento'] = df['Fechamento'].map(lambda t: float(t))
df['Variacao'] = df['Variacao'].map(lambda t: t.replace(",", "."))
df['Variacao'] = df['Variacao'].map(lambda t: float(t))
df['Variacao_%'] = df['Variacao_%'].map(lambda t: t.replace("%", ""))
df['Variacao_%'] = df['Variacao_%'].map(lambda t: t.replace(",", "."))
df['Variacao_%'] = df['Variacao_%'].map(lambda t: float(t))
df['Abertura'] = df['Abertura'].map(lambda t: t.replace(",", "."))
df['Abertura'] = df['Abertura'].map(lambda t: float(t))
df['Maxima'] = df['Maxima'].map(lambda t: t.replace(",", "."))
df['Maxima'] = df['Maxima'].map(lambda t: float(t))
df['Minima'] = df['Minima'].map(lambda t: t.replace(",", "."))
df['Minima'] = df['Minima'].map(lambda t: float(t))


In [28]:
df = df.to_csv('Dados_testes_da_VALE3.csv')

In [27]:
df

,Data,Fechamento,Variacao,Variacao_%,Abertura,Maxima,Minima
0,27 Abr 2022,82.28,4.63,5.96,80.46,82.70,80.15
1,26 Abr 2022,77.65,-1.63,-2.06,79.08,79.32,77.65
2,25 Abr 2022,79.28,-1.04,-1.29,77.94,79.61,77.54
3,22 Abr 2022,80.32,-4.98,-5.84,83.25,83.40,80.00
4,21 Abr 2022,85.30,0.00,0.00,86.61,86.67,84.40
...,...,...,...,...,...,...,...
59,03 Fev 2022,85.75,-0.04,-0.05,86.30,86.78,85.32
60,02 Fev 2022,85.79,0.48,0.56,85.50,86.29,84.72
61,01 Fev 2022,85.31,4.44,5.49,81.50,85.33,81.37
62,31 Jan 2022,80.87,-2.79,-3.33,82.94,82.94,80.87


In [45]:
def arrumar_link(acao, empresa):
    ''' 
        Função destinada a retornar se a ação é odinaria ou preferencial
        e fazendo o concatenação na função main
        Parametros:
        acao -> Tikcker 
        empresa -> nome da empresa na B3
        tipo -> on-ordinaria , pn-preferencial, unit-on e pn
        retorno -> string concatenada com variaveis empresa, acao e tipo
    '''
    
    # se nome de acao o final for 3 faça tipo recebe on #
    # se final do nome da acao for 4 tipo recebe pn #
    # se final de acao for igual a 11 então tipo recebe unit #
    if acao[-1:] == '3':
        tipo = 'on'
    elif acao[-1:] == '4' or acao[-1:]=='5' or acao[-1:]== '6':
        tipo = 'pn'
    elif acao[-2:] == '11':
        tipo = 'unit'
    # se tipo  for igual a unit então retorna sem o tipo #
    # caso contrario retorna empresa tipo e acao #
    if tipo == 'unit':
        return f'{empresa}-{acao}'
    else:
        return f'{empresa}-{tipo}-{acao}'
        

In [46]:
def retorna_int_href(acao, empresa):
    from time import sleep
    from selenium.webdriver.chrome.options import Options
    ''' 
        Função que retorna o numero de interações em que o for vai ser feito para a ação especifica
        comeco -> inteiro 
        final -> inteiro
        a -> string -> variavel que recebe o ajuste da função link 
        html -> Seletor 
        bs -> lista -> recebe todo o html
        linhas -> string -> aonde se encontra a estring que tem o valor maximo de tabelas para a interação
    '''
    comeco = 0
    final = 0
    a = arrumar_link(acao, empresa)
    url = f'https://br.advfn.com/bolsa-de-valores/bovespa/{a}/historico/mais-dados-historicos?current=0&Date1=03/02/17&Date2=04/05/22'
    option = Options()
    option.headless = True
    browser = webdriver.Chrome(options=option)
    sleep(2)
    browser.get(url)
    sleep(2)
    html = browser.page_source
    sleep(2)
    browser.close()
    bs = BeautifulSoup(html, 'html.parser')
    
    linhas = bs.find_all('a')[40]['href']
    # for dentro da string #
    # se linhas tiver '=' na string então comeco recebe o numero do array mais 1 #
    # se linhas na posição i  for igual a & então final recebe i que é o valor do array na posição aonde se encontra &#
    for i in range(len(linhas)):
        if linhas[i]== '=' :
            comeco = i+1
        elif linhas[i] == '&':
            final = i
            break
    # mostrar o link em linhas que nos intereça #
    #print(linhas)
    return int(linhas[comeco:final])

In [47]:
def adjustando_dataFrame(df):
    from datetime import datetime as dt
    from tqdm import tqdm
    #import warnings
    #warnings.filterwarnings("ignore")
    
    ''' 
        df -> Dataframe  aonde tem todos os valores puxados do webscraping 
        df['Fechamento'] -> valor de fechamento da acao
    '''
    
    df['Fechamento'] = df['Fechamento'].map(lambda t: t.replace(',', '.'))
    df['Fechamento'] = df['Fechamento'].map(lambda t: float(t))
    df['Variacao'] = df['Variacao'].map(lambda t: t.replace(",", "."))
    df['Variacao'] = df['Variacao'].map(lambda t: float(t))
    df['Variacao_%'] = df['Variacao_%'].map(lambda t: t.replace("%", ""))
    df['Variacao_%'] = df['Variacao_%'].map(lambda t: t.replace(",", "."))
    df['Variacao_%'] = df['Variacao_%'].map(lambda t: float(t))
    df['Abertura'] = df['Abertura'].map(lambda t: t.replace(",", "."))
    df['Abertura'] = df['Abertura'].map(lambda t: float(t))
    df['Maxima'] = df['Maxima'].map(lambda t: t.replace(",", "."))
    df['Maxima'] = df['Maxima'].map(lambda t: float(t))
    df['Minima'] = df['Minima'].map(lambda t: t.replace(",", "."))
    df['Minima'] = df['Minima'].map(lambda t: float(t))
    
    # transforma a data em string e arruma aonde tem Jan para /01/ etc ...#
    #df['Data'] = df['Data'].map(lambda x: str(x))
    df['Data'] = df['Data'].map(lambda x: x.replace('Jan', '/01/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Fev', '/02/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Mar', '/03/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Abr', '/04/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Mai', '/05/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Jun', '/06/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Jul', '/07/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Ago', '/08/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Set', '/09/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Out', '/10/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Nov', '/11/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Dez', '/12/'))
    # for  que intera dentro de data e altera todos os valores  para datas # 
    for i in tqdm(range(len(df['Data']))):
        df['Data'][i] = dt.strptime(df['Data'][i], '%d /%m/ %Y').date()  
    # arruma os valores em crescente no dataframe em relação a data fechamento, variacao, variacao a porcentagem, ... #    
    df = df.sort_values(by=['Data','Fechamento', 'Variacao', 'Variacao_%', 'Abertura', 'Maxima', 'Minima'])    
    # indexa a data como o index do dataframe #
    df = df.set_index('Data', drop=True)
    return df

In [48]:
def import_test(acao, empresa):
    from tqdm import tqdm
    from pandas import DataFrame
    from selenium import webdriver
    from time import sleep
    from bs4 import BeautifulSoup
    from selenium.webdriver.chrome.options import Options
    ''' 
        acao -> string -> acao digitada pelo usuario
        empresa -> string -> nome da empresa
        num -> int -> inteiro que é recebido pela funçao retorna_int
        a -> string -> recebe uma 
        data -> lista -> que recebe os textos da string html 
        datanew -> lista -> que organiza limpa os dados da lista anterior 
        Days, Close, Variation, Varietion_p, Open, Max, Min -> lista -> que separa cada valor do webscraping
        df -> Dataframe -> aonde recebe as listas 
    '''
    Days, Close, Variation, Variation_p, Open, Max, Min = [],[],[],[],[],[],[]
    empresa = empresa.strip().lower().replace(' ', '-')
    a = arrumar_link(acao, empresa)
    num = retorna_int_href(acao, empresa)
    for i in tqdm(range(num, -1, -1)):
        url2 = f'http://br.advfn.com/bolsa-de-valores/bovespa/{a}/historico/mais-dados-historicos?current={i}&Date1=28/01/17&Date2=28/04/22'
        sleep(2)
        option = Options()
        sleep(2)
        option.headless = True
        sleep(2)
        browser = webdriver.Chrome(options=option)
        sleep(2)
        browser.get(url2)
        sleep(2)
        html = browser.page_source
        sleep(2)
        browser.close()
        bs = BeautifulSoup(html, 'html.parser')
        linhas = bs.find_all('tr', attrs={'class': 'result'})
        data = []
        j = 0
        while j < len(linhas):
            data.append(linhas[j].text)
            j += 1
        k = 0
        datanew = []
        while k < len(data):
            datanew.append(data[k].split('\n'))
            k += 1
        l = 1
        while l < len(datanew):
            Days.append(datanew[l][1])
            Close.append(datanew[l][2])
            Variation.append(datanew[l][3])
            Variation_p.append(datanew[l][4])
            Open.append(datanew[l][5])
            Max.append(datanew[l][6])
            Min.append(datanew[l][7])
            l += 1
            
    df = DataFrame({
            'Data': Days,
            'Fechamento': Close,
            'Variacao': Variation,
            'Variacao_%': Variation_p,
            'Abertura': Open,
            'Maxima': Max,
            'Minima': Min,
    })
    
    df = adjustando_dataFrame(df)
    
    df.to_csv(f'{acao}-{empresa}.csv')
    
    return df
        

In [49]:
acao = 'YDUQ3'
empresa = 'estacio part'

In [50]:
df2 = import_test(acao, empresa)

 85%|████████▍ | 11/13 [09:34<01:44, 52.19s/it]


TimeoutException: Message: timeout: Timed out receiving message from renderer: 297.696
  (Session info: headless chrome=101.0.4951.64)
Stacktrace:
Backtrace:
	Ordinal0 [0x01217413+2389011]
	Ordinal0 [0x011A9F61+1941345]
	Ordinal0 [0x0109C658+837208]
	Ordinal0 [0x0108DB80+777088]
	Ordinal0 [0x0108C995+772501]
	Ordinal0 [0x0108CF98+774040]
	Ordinal0 [0x0109796F+817519]
	Ordinal0 [0x010A243B+861243]
	Ordinal0 [0x010A4E10+871952]
	Ordinal0 [0x0108D266+774758]
	Ordinal0 [0x010A2289+860809]
	Ordinal0 [0x010F464D+1197645]
	Ordinal0 [0x010E3F66+1130342]
	Ordinal0 [0x010BE546+976198]
	Ordinal0 [0x010BF456+980054]
	GetHandleVerifier [0x013C9632+1727522]
	GetHandleVerifier [0x0147BA4D+2457661]
	GetHandleVerifier [0x012AEB81+569713]
	GetHandleVerifier [0x012ADD76+566118]
	Ordinal0 [0x011B0B2B+1968939]
	Ordinal0 [0x011B5988+1989000]
	Ordinal0 [0x011B5A75+1989237]
	Ordinal0 [0x011BECB1+2026673]
	BaseThreadInitThunk [0x755BFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77287A7E+286]
	RtlGetAppContain

In [21]:
df2

NameError: name 'df2' is not defined

In [51]:
df2 = df2.to_csv(f'{acao}_{empresa}.csv')

In [32]:
acao = 'HAPV3'
empresa = 'hapvida'
tipo = 'on'

In [29]:
a = retorna_int_href(acao, empresa, tipo)

In [30]:
print(a)

16


In [20]:
for i,v in enumerate(a):
    print(i , v)

0 <a href="/" id="advfn-logo" title="Cotações de Ações da Bolsa de Valores Bovespa">
<img alt="ADVFN Brasil" border="0" height="58" src="/common/images/logos/br.advfn.com.gif" width="176"/>
</a>
1 <a href="/p.php?pid=prodmatrix&amp;xref=header_upgrade_text" id="prod-matrix-br">Produtos ADVFN</a>
2 <a href="/fale-conosco" id="prod-matrix-br">Fale Conosco</a>
3 <a class="header-btn" href="//br.advfn.com/common/account/register?xref=header_regBtn">Cadastro Gratis</a>
4 <a class="header-btn" href="//br.advfn.com/common/account/login" role="login">Login</a>
5 <a href="/forum">Forum</a>
6 <a href="/p.php?pid=level2" onclick="return setJavaLink(this,'BOV:HAPV3');">Book</a>
7 <a href="/jornal">Notícias</a>
8 <a href="/conversor-moedas">Moeda</a>
9 <a href="/mercados/bov/regular">Rankings</a>
10 <a href="/ferramentas/rastreador" title="Scanner">Scanner</a>
11 <a href="/mundo">Bolsas</a>
12 <a href="/common/plusonecoin/educacional" title="">
<img alt="PlusOneCoin" src="/common/plusonecoin/images